In [18]:
# # Warning control
# import warnings
# warnings.filterwarnings('ignore')

In [19]:
from crewai import Agent, Task, Crew

In [21]:
import json
with open("../../../api_keys.json", mode = 'r') as f:
    api_keys = json.load(f)

In [59]:
import os
# from utils import get_openai_api_key

# openai_api_key = get_openai_api_key()
os.environ["GOOGLE_API_KEY"] = api_keys['ashishjain1547'] 
os.environ["MODEL"] = "gemini/gemini-2.0-flash"

In [60]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [61]:
import litellm

litellm.api_key = api_keys['ashishjain1547']  # Ensure this is set
litellm.model = "gemini/gemini-2.0-flash"  # Ensure Gemini is being used

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [62]:
grammar_expert = Agent(
    role="Hindi-English Grammar Expert",
    goal="Evaluate a Hindi-English sentence pair for grammatical correctness."
    "Hindi: {hindi}\nEnglish: {english}",
    backstory="You're working on a language learning app for novice English learner who are fluent in Hindi."
              "You evaluate sentences for grammatical correctness."
              "You return your answer as 0 if sentence pair are grammatically incorrect and 1 if sentence pair are grammatically correct."
              "You give 0 also in the case if either English or Hindi sentence alone is grammatically incorrect.",
    allow_delegation=False,
	verbose=True
)

In [63]:
profanity_checker = Agent(
    role="Profanity Checker",
    goal="Evaluate a Hindi-English sentence pair for profanity"
    "Hindi: {hindi}\nEnglish: {english}",
    backstory="You're working on a language learning app for novice English learner who are fluent in Hindi."
    "You evaluate sentences for profanity."
    "If the sentence (or both sentences) are double meaning, directly or indirectly vulgar, or contain profanity, you return 0."
    "Otherwise, you return 1.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

In [64]:
check_grammar = Task(
    description=(
        "1. Check the grammar of the Hindi sentence.\n"
        "2. Check the grammar of the English sentence.\n"
        "3. Check that Hindi-English sentence pair mean the same thing."
    ),
    expected_output="0 if grammar is incorrect, and 1 if grammar is correct",
    agent=grammar_expert
)

### Task: Write

In [65]:
profanity_check = Task(
    description=(
        "1. Check the Hindi sentence for profanity, vulgarness or double meaning.\n"
        "2. Check the English sentence for profanity, vulgarness or double meaning."
    ),
    expected_output="0 if profanity is there, and 1 if profanity is NOT there",
    agent=profanity_checker
)

### Task: Edit

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [66]:
crew = Crew(
    agents=[grammar_expert, profanity_checker],
    tasks=[check_grammar, profanity_check],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [67]:
result = crew.kickoff(inputs={"hindi": "मैं दूध हूँ", "english": "I am milk"})

 
[2025-03-13 20:34:12][🚀 CREW 'CREW' STARTED, 9DF17E85-39E2-4958-876A-BDB3201F8447]: 2025-03-13 20:34:12.712794
 
[2025-03-13 20:34:12][📋 TASK STARTED: 1. CHECK THE GRAMMAR OF THE HINDI SENTENCE.
2. CHECK THE GRAMMAR OF THE ENGLISH SENTENCE.
3. CHECK THAT HINDI-ENGLISH SENTENCE PAIR MEAN THE SAME THING.]: 2025-03-13 20:34:12.733530
 
[2025-03-13 20:34:12][🤖 AGENT 'HINDI-ENGLISH GRAMMAR EXPERT' STARTED TASK]: 2025-03-13 20:34:12.735223
# Agent: Hindi-English Grammar Expert
## Task: 1. Check the grammar of the Hindi sentence.
2. Check the grammar of the English sentence.
3. Check that Hindi-English sentence pair mean the same thing.
 
[2025-03-13 20:34:12][🤖 LLM CALL STARTED]: 2025-03-13 20:34:12.735502
 
[2025-03-13 20:34:15][✅ LLM CALL COMPLETED]: 2025-03-13 20:34:15.229963


# Agent: Hindi-English Grammar Expert
## Final Answer: 
0


 
[2025-03-13 20:34:15][✅ AGENT 'HINDI-ENGLISH GRAMMAR EXPERT' COMPLETED TASK]: 2025-03-13 20:34:15.231743
 
[2025-03-13 20:34:15][✅ TASK COMPLETED: 1. 

- Display the results of your execution as markdown in the notebook.

In [70]:
for task in result.tasks_output:
    print(f"Agent: {task.agent}")
    print(f"Description: {task.description}")
    print(f"Expected Output: {task.expected_output}")
    print(f"Task Result: {task.raw}\n")

Agent: Hindi-English Grammar Expert
Description: 1. Check the grammar of the Hindi sentence.
2. Check the grammar of the English sentence.
3. Check that Hindi-English sentence pair mean the same thing.
Expected Output: 0 if grammar is incorrect, and 1 if grammar is correct
Task Result: 0

Agent: Profanity Checker
Description: 1. Check the Hindi sentence for profanity, vulgarness or double meaning.
2. Check the English sentence for profanity, vulgarness or double meaning.
Expected Output: 0 if profanity is there, and 1 if profanity is NOT there
Task Result: 1

